# Pruebas de hipotesis y validación

## Importar librerias

In [1]:
import numpy as np
import pandas as pd
import random
import io

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

## Cargar base de datos

![petalo-sepalo](img/petalo-sepalo.jpg)

In [2]:
df = pd.read_csv('../data/iris.csv')

df.drop(axis=1, columns='Id', inplace=True)

nuevos_nombres = {
    'SepalLengthCm' : 'sepal_length_cm', 
    'SepalWidthCm' : 'sepal_width_cm', 
    'PetalLengthCm' : 'petal_length_cm', 
    'PetalWidthCm': 'petal_width_cm',
    'Species' : 'species'
}

df = df.rename(columns=nuevos_nombres)

df.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Tipos de pruebas

### Distribución t de Student

Standard error

$$SE = \frac{\sigma }{\sqrt{N}}

In [3]:
se_length = df['sepal_length_cm'].std()/np.sqrt(10)
se_width = df['sepal_width_cm'].std()/np.sqrt(10)

print(se_length)
print(se_width)

0.2618575017646526
0.13711456043966944


Standard error diviation

$$SED = \sqrt{\frac{\sigma_{1}^{2}}{N_{1}}+\frac{\sigma_{1}^{2}}{N_{2}}}

In [4]:
sed = np.sqrt((se_length**2)+(se_width**2))
print(sed)

0.2955837511010861


t

$$ t = \frac{(\bar{x}_{1}-\bar{x}_{2})}{SED}

In [5]:
t_stat = (df['sepal_length_cm'].mean()-df['sepal_width_cm'].mean())/sed
print(t_stat)

9.436693738890318


In [6]:
p = ttest_ind(df['sepal_length_cm'], df['sepal_width_cm'])
print(p)

TtestResult(statistic=36.548157693982006, pvalue=3.987838114848222e-112, df=298.0)


### Correlación y coeficiente de Pearson

In [7]:
df['sepal_length_cm'].corr(df['sepal_width_cm'])

-0.10936924995064937

In [8]:
st.pearsonr(df['sepal_length_cm'], df['sepal_width_cm'])

PearsonRResult(statistic=-0.10936924995064937, pvalue=0.18276521527136963)

### Analisis de la varianza (ANOVA)

In [9]:
f_oneway(df['sepal_length_cm'],df['sepal_width_cm'])

F_onewayResult(statistic=1335.7678308241748, pvalue=3.98783811484836e-112)

## Bootstraping

In [10]:
data = np.random.normal(loc=34, size=10000) # poblacion de 10.000 con una media de 34 años
data

array([32.97082875, 33.53378013, 34.29026957, ..., 33.35880468,
       34.31336632, 34.46101601])

In [11]:
data.mean()

33.98607741565325

### Calculando el promedio de edades

In [12]:
promedio = []
for i in range(40):
    muestra = random.sample(data.tolist(),5)
    prom = np.mean(muestra)
    promedio.append(prom)

In [13]:
promedio

[33.6668836131165,
 33.76391542535837,
 33.712729325427865,
 33.99368882500079,
 33.38843671324783,
 33.777789878357154,
 33.81368480603403,
 33.1485843864026,
 33.71601229947531,
 34.21493282399564,
 33.3763488419321,
 34.26820374312197,
 33.68521044852762,
 34.427268229135855,
 33.44556850274773,
 33.64313854290275,
 33.9799565876865,
 33.537660967420045,
 34.417458993997386,
 34.19117948730037,
 33.932158602197504,
 34.12176903980351,
 33.707976853150896,
 33.87849085866245,
 33.869691816732356,
 34.60380719591473,
 33.89930467894264,
 33.90700175058203,
 33.584222225056635,
 34.17303809501853,
 33.82633556190926,
 34.266140852638884,
 34.2817180682942,
 34.10165484308375,
 33.15333351047131,
 34.08174814660659,
 34.024940455972065,
 34.00715878126903,
 34.00642851115295,
 34.17515934874869]

In [14]:
np.mean(promedio)

33.89426829093491

## Validación cruzada

### Cargar base de datos

In [15]:
df = pd.read_csv('../data/breast-cancer-wisconsin.csv')
df.drop(axis=1, columns='Unnamed: 32', inplace=True)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [17]:
X = df.iloc[:,2:]
Y = df.iloc[:,1]

In [18]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [20]:
Y.value_counts()

diagnosis
B    357
M    212
Name: count, dtype: int64

In [21]:
# Partiendo entre train y tes
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=4)

In [22]:
# Modelo
modelo = DecisionTreeClassifier()
# Ajuste del modelo
modelo.fit(X_train, Y_train)
# Precisioòn del modelo en %
resultado = modelo.score(X_test, Y_test)
print(resultado)

0.9122807017543859


### Validación cruzada k fold

In [23]:
# Modelo
modelo = DecisionTreeClassifier()
# Muestras
kfold_validacion = KFold(10)

In [24]:
# Resultados
resultados = cross_val_score(modelo, X, Y, cv=kfold_validacion)
print(resultados)
print(resultados.mean())

[0.92982456 0.9122807  0.89473684 0.94736842 0.94736842 0.96491228
 0.89473684 0.98245614 0.92982456 0.94642857]
0.9349937343358397
